## get data index

In [1]:
import json
# read apikey from api_key.json
with open('api_key.json') as f:
    api_key = json.load(f)["api"]
# https://github.com/openai/openai-python
import openai
# Set up the OpenAI API client
openai.api_key = api_key

In [2]:
import mat73
path = "thesis-master/authorData/authorshipData.mat"
import os

In [3]:
# read mat file from path
data_dict = mat73.loadmat(path)
data_dict.keys()

dict_keys(['all_authors', 'all_freqs', 'all_wans', 'function_words'])

In [4]:
authors = data_dict['all_authors']
authors

[['abbott'],
 ['stevenson'],
 ['alcott'],
 ['alger'],
 ['allen'],
 ['austen'],
 ['bronte'],
 ['cooper'],
 ['dickens'],
 ['garland'],
 ['hawthorne'],
 ['james'],
 ['melville'],
 ['page'],
 ['thoreau'],
 ['twain'],
 ['doyle'],
 ['irving'],
 ['poe'],
 ['jewett'],
 ['wharton']]

In [5]:


# Possible authors:
# from file \graph-neural-networks-master\alegnn\utils\dataTools.py
all_authors_full = """jacob 'abbott',         robert louis 'stevenson',   louisa may 'alcott',
horatio 'alger',        james 'allen',              jane 'austen',
emily 'bronte',         james 'cooper',             charles 'dickens', 
hamlin 'garland',       nathaniel 'hawthorne',      henry 'james',
herman 'melville',      'page',                     henry 'thoreau',
mark 'twain',           arthur conan 'doyle',       washington 'irving',
edgar allan 'poe',      sarah orne 'jewett',        edith 'wharton'"""

# remove "\n"
all_authors_full = all_authors_full.replace("\n","")
# convert to list
all_authors = all_authors_full.split(",")
# remove all start spaces
all_authors = [author.strip() for author in all_authors]
# remove all '
authors_fullname = [author.replace("'","") for author in all_authors]
authors_fullname


['jacob abbott',
 'robert louis stevenson',
 'louisa may alcott',
 'horatio alger',
 'james allen',
 'jane austen',
 'emily bronte',
 'james cooper',
 'charles dickens',
 'hamlin garland',
 'nathaniel hawthorne',
 'henry james',
 'herman melville',
 'page',
 'henry thoreau',
 'mark twain',
 'arthur conan doyle',
 'washington irving',
 'edgar allan poe',
 'sarah orne jewett',
 'edith wharton']

In [6]:
def words_count(text):
    return len(text.split())

In [8]:


def excerpts_generator(author_name,word_count, model= "gpt-3.5-turbo"):
    #???? not sure, but if too many messages, maybe use acreate
    ### ??? find some bugs, chatgpt did not imitate them
    # Set up prompt
    message = f"Create a passage in the style of the novelist {author_name}, use about {word_count} words"
    messages=[{"role": "user", "content": message}]

    # Generate a response
    completion = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        max_tokens=1024,
        n=1,
        stop=None,
        temperature=0.5,
    )

    response = completion.choices[0].message.content
    return response

# response = excerpts_generator(authors_fullname[0], 500)
# response

In [60]:
responses = []
for author in authors_fullname:
    response = excerpts_generator(author, 500)
    responses.append(response)

# create a json file to store the responses
import json
with open('responses.json', 'w') as f:
    # key is author name, value is the response
    for i in range(len(authors_fullname)):
        json.dump({authors_fullname[i]:responses[i]}, f)
        f.write('\n')
    

In [73]:
# use map to apply the function to all authors
_words_count = list(map(words_count, responses))
_words_count

[474,
 605,
 408,
 466,
 540,
 617,
 501,
 510,
 464,
 522,
 607,
 505,
 465,
 612,
 491,
 572,
 513,
 788,
 481,
 429,
 620]

In [66]:
# read the responses from json file
with open('responses.json') as f:
    responses = [json.loads(line) for line in f]
# get every value from the dictionary
responses = [list(response.values())[0] for response in responses]

In [67]:
responses

['It was a bright and sunny day in the small town of Millfield. The birds chirped merrily in the trees, and the gentle breeze rustled the leaves. As I walked down the main street, I couldn\'t help but marvel at the quaint charm of the place. The houses were all made of wood, with white picket fences and well-manicured lawns. The people of Millfield were friendly and welcoming, always ready with a smile and a kind word.\n\nAs I passed by the town square, I noticed a group of children playing tag. They were laughing and running around with abandon, their youthful energy infectious. It reminded me of my own childhood, of the simpler times when life was carefree and full of wonder.\n\nFurther down the street, I came across a small bookstore. The sign on the door read "Books and More," and I couldn\'t resist the temptation to step inside. The shelves were lined with books of all genres, from classic literature to modern-day thrillers. The scent of old paper and ink filled the air, and I fel

In [10]:
# try for gpt 4
# cannot use now


# model = "gpt-4"

# responses = []
# for author in authors_fullname:
#     response = excerpts_generator(author, 500,model= model)
#     responses.append(response)

# # create a json file to store the responses
# import json
# with open('responses_gpt4.json', 'w') as f:
#     # key is author name, value is the response
#     for i in range(len(authors_fullname)):
#         json.dump({authors_fullname[i]:responses[i]}, f)
#         f.write('\n')

## test

In [ ]:
# try for gpt 4
# cannot use now


# model = "gpt-4"

# responses = []
# for author in authors_fullname:
#     response = excerpts_generator(author, 500,model= model)
#     responses.append(response)

# # create a json file to store the responses
# import json
# with open('responses_gpt4.json', 'w') as f:
#     # key is author name, value is the response
#     for i in range(len(authors_fullname)):
#         json.dump({authors_fullname[i]:responses[i]}, f)
#         f.write('\n')

In [ ]:
# ask chatgpt to generate topic for a given author
def generate_topic(author, num_titles=10, model= "gpt-3.5-turbo"):
    message = f"Please give me {num_titles} titles of poems that {author[0]} has created, Please indicate with {{}} and separate with a comma"
    messages=[{"role": "user", "content": message}]
    print(messages)
    # Generate a response
    completion = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        max_tokens=1024,
        n=1,
        stop=None,
        temperature=0.5,
    )

    response = completion.choices[0].message.content
    return response
generate_topic(authors[4])

[{'role': 'user', 'content': 'Please give me 10 titles of poems that allen has created, Please indicate with {} and separate with a comma'}]


'{Howl}, {America}, {A Supermarket in California}, {Sunflower Sutra}, {Kaddish}, {Song}, {To Aunt Rose}, {Mescaline}, {Wichita Vortex Sutra}, {The Ballad of the Skeletons}'

In [5]:


def excerpts_generator(prompt,word_count, model= "gpt-3.5-turbo"):
    #???? not sure, but if too many messages, maybe use acreate
    
    # Set up prompt
    message = f"please write a poem with topic {prompt}, use about {word_count} words"
    messages=[{"role": "user", "content": message}]

    # Generate a response
    completion = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        max_tokens=1024,
        n=1,
        stop=None,
        temperature=0.5,
    )

    response = completion.choices[0].message.content
    return response



In [ ]:


def excerpts_generator(author_name,word_count, model= "gpt-3.5-turbo"):
    #???? not sure, but if too many messages, maybe use acreate
    
    # Set up prompt
    message = f"Create a passage in the style of the novelist {author_name}, use about {word_count} words"
    messages=[{"role": "user", "content": message}]

    # Generate a response
    completion = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        max_tokens=1024,
        n=1,
        stop=None,
        temperature=0.5,
    )

    response = completion.choices[0].message.content
    return response

name = "nathaniel hawthorne"

In [6]:
topic = "rain"
word_count = 100
poem = poem_generator(topic,word_count)
# count words
print(poem)
print(len(poem.split(" ")))

Rain, oh rain, how you fall so free,
Bringing life to all that you see.
Your gentle drops, a soothing sound,
As you wash the earth, and all around.

With every drop, you cleanse the air,
And wash away the world's despair.
You fill the rivers, and the streams,
And bring to life, the world of dreams.

The pitter-patter of your feet,
A symphony of nature's beat.
And as you fall, you bring new life,
A world renewed, without strife.

So let it rain, and let it pour,
For in your wake, we all adore.
The beauty that you bring each day,
A world refreshed, in every way.
93


In [ ]:
class GPT_writter():
    def __init__(self,topic,word_count):
        self.topic = topic
        self.word_count = word_count
        self.poem = poem_generator(topic,word_count)
        self.poem_len = len(self.poem.split(" "))
        
    def data_fetch(self):
        pass
    
    def poem_generator(prompt,word_count, model_engine= "gpt-3.5-turbo"):
        # Set up the model and prompt
        message = f"please write a poem with topic {prompt}, use about {word_count} words"
        
        messages=[{"role": "user", "content": message}]

        # Generate a response
        completion = openai.ChatCompletion.create(
            model=model_engine,
            messages=messages,
            max_tokens=1024,
            n=1,
            stop=None,
            temperature=0.5,
        )

        response = completion.choices[0].message.content
        return response